In [1]:
!pip install langchain
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install anthropic
!pip install fpdf
!pip install pypdf2



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.summarize import load_summarize_chain
from langchain.schema import Document
from langchain.chat_models import ChatOpenAI
from langchain import PromptTemplate
import tiktoken
import PyPDF2


In [3]:
OPENAI_API_KEY = ""

In [15]:
def loadFile(filename):
    """Load and extract text from each page of a PDF file using `PdfReader`."""
    with open(filename, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        pages = [page.extract_text() for page in reader.pages]
    return pages

def combine_docs(docs):
    """Combine a list of documents (strings) into a single string."""
    return " ".join(docs)

def tokenize(text, encoding_name):
    # Placeholder function for tokenization to simulate encoding.
    # Replace this with your actual tiktoken.get_encoding(encoding_name).encode(text)
    return text.split()

def num_tokens_from_documents(docs, encoding_name="r50k_base"):
    """Returns the number of tokens from the list of documents."""
    text = combine_docs(docs)
    # Call your actual encoding/tokenization method here.
    num_tokens = len(tokenize(text, encoding_name))
    return num_tokens

In [16]:
num_tokens = num_tokens_from_documents(pages)

pages = loadFile('crime-and-punishment.pdf')
num_tokens = num_tokens_from_documents(pages)
print(f"Number of tokens: {num_tokens}")

Number of tokens: 213327


In [17]:
def chunk_documents(text:str, chunk_size: int, overlap: int):
  """Returns the chunked documents from the text"""
  splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = overlap,
    separators=["\n\n", "\n", "\t"],
    keep_separator=True)
  return splitter.create_documents([text])

# The chunked text will have 10000 characters each.
chunked_text = chunk_documents(combine_docs(pages),10000,1000)

In [18]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectors = embeddings.embed_documents([x.page_content for x in chunked_text])

d:\Programs\Python\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.0.9 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [19]:
# Reducing the number of tokens by removing the '\n' characters since these characters do not convey any meaning
for doc in chunked_text:
  doc.page_content = doc.page_content.replace('\n','')

In [20]:

map_prompt = """
You will be given a single passage of a book. This section will be enclosed in triple backticks (```)
Your goal is to give a summary of this section so that a reader will have a full understanding of what happened.
Your response should fully encompass what was said in the passage. Create a comprehensive summary in paragraph form
DO NOT CREATE BULLETE POINTS

```{text}```
FULL SUMMARY:
"""
map_prompt_template = PromptTemplate(template=map_prompt, input_variables=["text"])

In [21]:
llm = ChatOpenAI(model='gpt-4-1106-preview', api_key=OPENAI_API_KEY)

d:\Programs\Python\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [22]:
map_chain = load_summarize_chain(llm=llm,
                             chain_type="stuff",
                             prompt=map_prompt_template)


In [23]:
# Make an empty list to hold your summaries
summary_list = []

# Loop through a range of the lenght of your selected docs
for i, doc in enumerate(chunked_text):

    chunk_summary = map_chain.run([doc])

    summary_list.append(chunk_summary)

    print (f"Summary #{i} (chunk #{chunked_text[i]}) - Preview: {chunk_summary[:250]} \n")


d:\Programs\Python\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Summary #0 (chunk #page_content='Download free eBooks of classic literature, books and novels at Planet eBook. Subscribe to our free eBooks blog and email newsletter.Crime and PunishmentBy Fyodor Dostoevsky Crime and Punishment \x18Translator’s PrefaceA few words about Dostoevsky himself may help the Eng -lish reader to understand his work.Dostoevsky was the son of a doctor. His parents were very hard- working and deeply religious people, but so poor that they lived with their five children in only two rooms. The father and mother spent their evenings in reading aloud to their children, generally from books of a serious charac -ter.Though always sickly and delicate Dostoevsky came out third in the final examination of the Petersburg school of Engineering. There he had already begun his first work, ‘Poor Folk.’This story was published by the poet Nekrassov in his review and was received with acclamations. The shy, un -known youth found himself instantly something of a celebrity. A brill

KeyboardInterrupt: 

In [25]:
summaries = "\n".join(summary_list)

# Convert it back to a document
summaries = Document(page_content=summaries)

print (f"Your total summary has {llm.get_num_tokens(summaries.page_content)} tokens")

Your total summary has 1767 tokens


In [26]:
combine_prompt = """
You are given multiple summaries from a book enclosed in tripple back ticks. The summaries are generated from multiple sections of the book.
Your job now is to write a comprehensive summary of the book after understanding the input text.
The final answer should be a detailed summary of the book and should be detailed. Make it long and detailed.
Use the tone of an experience book reviewer.
```{text}```
SUMMARY:
"""
combine_prompt_template = PromptTemplate(template=combine_prompt, input_variables=["text"])

In [27]:
reduce_chain = load_summarize_chain(llm=llm,
                             chain_type="stuff",
                             prompt=combine_prompt_template,
                              verbose=True # Set this to true if you want to see the inner workings
                                   )

In [28]:
output = reduce_chain.run([summaries])



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are given multiple summaries from a book enclosed in tripple back ticks. The summaries are generated from multiple sections of the book.
Your job now is to write a comprehensive summary of the book after understanding the input text.
The final answer should be a detailed summary of the book and should be detailed. Make it long and detailed.
Use the tone of an experience book reviewer.
```The passage provides an introduction to Fyodor Dostoevsky's background and his novel "Crime and Punishment." Dostoevsky, the son of a doctor, grew up in poverty with his deeply religious parents and siblings in a two-room dwelling. Despite being sickly, he excelled academically, finishing third in his class at the Petersburg School of Engineering and writing his first work, "Poor Folk." His early promise was disrupted when he was arrested in 1849 for engaging in anti-censorship discussions and

In [29]:
# Storing the generate summary in PDF file
from fpdf import FPDF

def create_pdf(my_string, file_path):
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", size=11)

    # Split the string into paragraphs
    paragraphs = my_string.split('\n')

    for paragraph in paragraphs:
        pdf.multi_cell(0, 5, paragraph)
        pdf.ln(2)

    # Save the PDF to the specified file path
    pdf.output(file_path)

    print(f"PDF saved to {file_path}")


create_pdf(output,'summary.pdf')

UnicodeEncodeError: 'latin-1' codec can't encode character '\u2014' in position 397: ordinal not in range(256)